In [1]:
import sys 
sys.path.append('/workspace/Documents')
import os
import torch
import numpy as np
import nibabel as nb
import Example_UNet.model.model as model
import Example_UNet.functions_collection as ff
import Example_UNet.Build_lists.Build_list as Build_list
import Example_UNet.Generator as Generator

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 1: define pre-trained model

In [10]:
trial_name = 'trial'
epoch = 2  # picked the epoch of a trained model
trained_model_filename = os.path.join('/mnt/camca_NAS/denoising/models', trial_name, 'models/model-' + str(epoch)+ '.pt') # path to trained model
save_folder = os.path.join('/mnt/camca_NAS/denoising/models', trial_name, 'pred_images'); os.makedirs(save_folder, exist_ok=True) # path to save predicted images

In [11]:
### parameters no need to change
image_size = [512,512]

histogram_equalization = True
background_cutoff = -1000
maximum_cutoff = 2000
normalize_factor = 'equation' 

### step 2: build patient list - testing, batch 5

In [12]:
patient_list_spreadsheet = os.path.join('/workspace/Documents/Example_UNet/example_data/Patient_lists/patient_list_example.xlsx') # change to your own path
build_sheet =  Build_list.Build(patient_list_spreadsheet)

_,patient_id_list, input_list, reference_list = build_sheet.__build__(batch_list = [1]) 

### step 3: build model

In [13]:
# build model
our_model = model.Unet2D(
    init_dim = 16,
    channels = 1, 
    out_dim = 1,
    dim_mults = (2,4,8,16),
    full_attn = (None,None, False, True),
    act = 'ReLU',
)

in out is :  [(16, 32), (32, 64), (64, 128), (128, 256)]


### step 4: main

In [14]:
# main
for i in range(0, patient_id_list.shape[0]):
    patient_id = patient_id_list[i]
    input_file = input_list[i]
    reference_file = reference_list[i]

    print(i,patient_id, input_file, reference_file)

    # get the reference image
    reference_img = nb.load(reference_file)
    affine = reference_img.affine; reference_img = reference_img.get_fdata()

    # make folders
    ff.make_folder([os.path.join(save_folder, patient_id)])
    save_folder_case = os.path.join(save_folder, patient_id, 'epoch' + str(epoch)); os.makedirs(save_folder_case, exist_ok=True)

    # # generator
    generator = Generator.Dataset_2D(
        input_list = np.asarray([input_file]),
        reference_list = np.asarray([reference_file]),

        image_size = image_size,

        num_slices_per_image = reference_img.shape[-1],
        random_pick_slice = False,
        slice_range = None,

        background_cutoff = background_cutoff,
        maximum_cutoff = maximum_cutoff,
        normalize_factor = normalize_factor,

        num_patches_per_slice = 1,
        patch_size = image_size, ## validation on full image

        shuffle = False,
        augment = False,
        augment_frequency = 0,)

    # # sample:
    sampler = model.Sampler(our_model,generator,batch_size = 1, image_size = image_size)

    pred_img = sampler.sample_2D(trained_model_filename, reference_img)
    pred_img_final = pred_img
    
    # save
    nb.save(nb.Nifti1Image(pred_img_final, affine), os.path.join(save_folder_case, 'pred_img.nii.gz'))

0 ID_002 /workspace/Documents/Example_UNet/example_data/data/ID_002/input.nii.gz /workspace/Documents/Example_UNet/example_data/data/ID_002/output_reference.nii.gz


/workspace/Documents/Example_UNet/model/model.py:750: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(trained_model_filename, map_location=self.device)


model device:  cuda:0
gt_img shape:  (512, 512, 50)
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and random origin y is:  0
random origin x is:  0  and ra